In [1]:
import os
import datetime
# Local Imports
from src.coastseg import coastseg_logs #must be the first module loaded to create logs folder
from src.coastseg import coastseg_map

# External Imports
import ee
from google.auth import exceptions as google_auth_exceptions
from IPython.display import display, clear_output
from tkinter import Tk,filedialog
from ipywidgets import Button, HBox, VBox, Layout, DatePicker, HTML, RadioButtons, SelectMultiple, Output, Checkbox

# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Authenticate and Initialize with Google Earth Engine (GEE)

- Run this cell to initialize with GEE which will allow you to download remote sensing data from GEE.

### First Time Users

- In order to use Google Earth Engine (GEE) you will need to sign up to request access to use Google Earth Engine.https://signup.earthengine.google.com. You will only need to do this once and it takes only a day to get your account verified.

### How `ee.Authenticate()` works

- In order to initialize with GEE you will need an authorization token with is obtained by running `ee.Authenticate()`.This token lasts 7 days and during those 7 days you will not need to authenticate with google earth engine with an access code. Once the 7 days are up you will need to reauthenticate to use GEE again.


In [2]:
try:
    ee.Initialize()
except google_auth_exceptions.RefreshError as exception:
    print("Please authenticate with Google:\n")
    ee.Authenticate()
    ee.Initialize()

In [3]:
# Date Widgets
start_date=DatePicker(
    description='Start Date',
    value=datetime.date(2018, 12, 1),
    disabled=False,
)
end_date=DatePicker(
    description='End Date',
    value=datetime.date(2019, 3, 1), #2019, 1, 1
    disabled=False,
)
date_instr=HTML(
    value="<b>Pick a date:</b>",
    layout=Layout(padding='10px')
)
dates_box=HBox([start_date,end_date])
dates_vbox=VBox([date_instr,dates_box])


collection_instr=HTML(
    value="<b>Pick a collection</b>\
        <br> C01: LandSat Collection 1 \
        <br> C02: LandSat Collection 2  [2022/01/01 and beyond] \
        <br>      - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)
collection_radio=RadioButtons(
    options=['C01', 'C02'],
    value='C01',
    description='Collection:',
    disabled=False
)
collection_vbox=VBox([collection_instr,collection_radio])

sat_instr=HTML(
    value="<b>Pick multiple satellites by holding the control key:</b> \
        <br> - images prior to 2022/01/01 will be downloaded from Collection 1 \
        <br> - images after 2022/01/01 will be automatically downloaded from Collection 2 \
        <br> C01: LandSat Collection 1 : All dates before 2022/01/01\
        <br> C02: LandSat Collection 2 : All dates after 2022/01/01 \
        <br> - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)


satellite_selection=SelectMultiple(
    options=['L5', 'L7', 'L8', 'S2'],
    value=['L8'],
    description='Satellites',
    disabled=False
)
# value=['S2'],
sat_vbox = VBox([sat_instr,satellite_selection])

def handle_collection_change(change):
    if change['new'] == 'C02':
        satellite_selection.options=['L5', 'L7', 'L8', 'L9', 'S2']
    else:
        satellite_selection.options=['L5', 'L7', 'L8', 'S2']

collection_radio.observe(handle_collection_change,"value")

controls_vbox=VBox([dates_vbox, collection_vbox, sat_vbox])
controls_vbox

In [4]:
# CoastSat Download Variables
# ------------------------------
# Save the satellites selected to sat_list
if satellite_selection.value:
    sat_list = list(satellite_selection.value)
elif not satellite_selection.value:
    print("ERROR: You must select at least one satellite first")
# Save the dates selected by the user as well as the selected collection
dates = [str(start_date.value),str(end_date.value)]
collection = collection_radio.value
print(f"dates: {dates}")
print(f"collection: {collection}")
print(f"sat_list: {sat_list}")

dates: ['2018-12-01', '2019-03-01']
collection: C01
sat_list: ['L8']


## Settings for CoastSat

Modify any of the following setting to determine how data is downloaded from CoastSat


In [5]:
pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
     'dist_clouds': 300,        # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off':'False',          # if True, no pan-sharpening is performed on Landsat 7,8 and 9 imagery
    'create_plot':False,        # True create a matplotlib plot of the image with the datetime as the title
    'max_dist_ref':25
}

# Create the Map

- Create the map object here
- Save all the settings created earlier
- Save the list of satellites, dates, and the collection to coastseg_map instance


In [6]:
coastsegmap=coastseg_map.CoastSeg_Map()

In [7]:
coastsegmap.save_settings(sat_list=sat_list, collection = collection,dates=dates,**pre_process_settings)

In [8]:
coastsegmap.settings

{'sat_list': ['L8'],
 'collection': 'C01',
 'dates': ['2018-12-01', '2019-03-01'],
 'cloud_thresh': 0.5,
 'dist_clouds': 300,
 'output_epsg': 3857,
 'check_detection': True,
 'adjust_detection': False,
 'save_figure': True,
 'min_beach_area': 4500,
 'buffer_size': 150,
 'min_length_sl': 200,
 'cloud_mask_issue': False,
 'sand_color': 'default',
 'pan_off': 'False',
 'create_plot': False,
 'max_dist_ref': 25}

# How to Use The Map

---

1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.

- You should see a coastline appear in yellow and some rectangles along it.

3. Click 1 or more ROIs then click `Save ROI` to save these ROIs for downloading later
4. You're ready to download the ROIs. Run the next block of code.


In [9]:
from src.coastseg.map_UI import UI
coastseg_ui = UI(coastsegmap)
coastseg_ui.create_dashboard()

In [ ]:
coastsegmap.settings

In [ ]:
coastsegmap.rois.master_config

In [ ]:
import geopandas as gpd
import numpy as np
file_path=r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data\ID02022-10-07__13_hr_15_min45sec\ID12022-10-07__13_hr_15_min45sec_output_lines_staticref.geojson'
with open(file_path, 'r') as f:
    gpd_data = gpd.read_file(f)
gpd_data = gpd_data.to_crs("epsg:4326")
gpd_data.to_file('static_shoreline.geojson',driver='GeoJSON')

In [ ]:
from shapely.geometry import Point
d = {'col1': ['name1', 'name2'], 'geometry': [Point(1, 2), Point(2, 1)]}
gpd.GeoDataFrame(d).to_file('HAHAA.geojson',driver='GeoJSON')

In [ ]:
gpd_data.crs

In [ ]:
gpd_data = gpd_data.to_crs("epsg:4326")
gpd_data

In [ ]:
print(gpd_data['geometry'][0])

In [ ]:
gpd_data.crs

In [ ]:
gpd_data.to_file('static_shoreline.geojson',driver='GeoJSON')

In [ ]:
with open(r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\ugh.geojson', 'r') as f:
    gpd_data = gpd.read_file(f)
gpd_data

In [ ]:
gpd_data.crs

In [ ]:
import os
import json
import logging
from typing import Union

from src.coastseg.bbox import Bounding_Box
from src.coastseg import common
from src.coastseg.shoreline import Shoreline
from src.coastseg.transects import Transects
from src.coastseg.roi import ROI
from src.coastseg import exceptions

import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

from coastsat import SDS_tools, SDS_download, SDS_tools,SDS_transects,SDS_shoreline, SDS_preprocess
from ipyleaflet import DrawControl, LayersControl,  WidgetControl, GeoJSON
from leafmap import Map
from ipywidgets import Layout, HTML, Accordion
from tqdm.auto import tqdm
from pyproj import Proj, transform

In [ ]:
settings_transects = { # parameters for computing intersections
                      'along_dist':          25,        # along-shore distance to use for computing the intersection
                      'min_points':          3,         # minimum number of shoreline points to calculate an intersection
                      'max_std':             15,        # max std for points around transect
                      'max_range':           30,        # max range for points around transect
                      'min_chainage':        -100,      # largest negative value along transect (landwards of transect origin)
                      'multiple_inter':      'max',    # mode for removing outliers ('auto', 'nan', 'max')
                      'prc_multiple':         0.1,      # percentage of the time that multiple intersects are present to use the max
                     }
settings_transects

In [ ]:
roi_id=16
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:'+str(coastsegmap.settings['output_epsg']))
cross_distance_max = 0
try:
    # Select a single roi by id
    single_roi = coastsegmap.rois.gdf[coastsegmap.rois.gdf['id']==roi_id]
    # if the id was not found in the geodataframe raise an exception
    if single_roi.empty:
        logger.error(f"Id: {id} was not found in {coastsegmap.rois.gdf}")
        raise Exception("Id: {id} was not found in {coastsegmap.rois.gdf}")
    
    extracted_shoreline=coastsegmap.rois.extracted_shorelines[int(roi_id)]
    # if no extracted shoreline exists for the roi's id then return cross distance = 0  
    if extracted_shoreline == {} :
         print({})
    
    # Get transects intersecting this specific ROI as a geodataframe
    transect_in_roi=coastsegmap.get_intersecting_transects(coastsegmap.rois.gdf,coastsegmap.transects.gdf,roi_id)
    
    print(f"compute_transects_for_roi() :transect_in_roi: {transect_in_roi}")
    # Do not compute the transect if no shoreline exists
    if not coastsegmap.is_shoreline_present(coastsegmap.rois.extracted_shorelines,int(roi_id)):
        raise exceptions.No_Extracted_Shoreline(f"No extracted shoreline at this roi {roi_id}")
    else:
        print("Shoreline present at ROI: ",roi_id)
        print("Creating transects...")
        print(f"compute_transects_for_roi() :Shoreline present at ROI: {roi_id}")
        # convert transects to lan,lon tuples 
        transects_coords = []
        for k in transect_in_roi['geometry'].keys():
            transects_coords.append(tuple(np.array(transect_in_roi['geometry'][k]).tolist()))
        print(f"transects_coords:{transects_coords}")
        # convert to dict of numpy arrays of start and end points
        transects = {}
        for counter,i in enumerate(transects_coords):
            x0,y0 = transform(inProj,outProj,i[0][0],i[0][1])
            x1,y1 = transform(inProj,outProj,i[1][0],i[1][1])    
            transects['NA'+str(counter)] = np.array([[x0,y0],[x1,y1]])
        print(f"compute_transects_for_roi():: transects: {transects}")
        # defines along-shore distance over which to consider shoreline points to compute median intersection (robust to outliers)
        if 'along_dist' not in coastsegmap.settings.keys():
            coastsegmap.settings['along_dist'] = 25 
        cross_distance_max = SDS_transects.compute_intersection(extracted_shoreline, transects, settings_transects ) 
        print(f"transects cross_distance:{cross_distance}")
except Exception as err:
    raise Exception(err)

cross_distance_max

In [ ]:
cross_distance_max

In [ ]:
cross_distance_standard

In [ ]:
#auto settings
cross_distance_auto

In [ ]:
import json
with open(f"ROI_{roi_id}_transects_cross_dis_auto.json",'w')as f:
    json.dump(cross_distance,f)

In [ ]:
coastsegmap.rois.cross_distance_transects

In [ ]:
coastsegmap.inputs_dict

In [ ]:
coastsegmap.rois.inputs_dict

In [ ]:
import json
with open("inputs.json",'w') as f:
    json.dump(coastsegmap.rois.inputs_dict,f)

In [ ]:
coastsegmap.shoreline.gdf

In [ ]:
coastsegmap.selected_ROI_layer.data

In [ ]:
import geopandas as gpd
import numpy as np
single_roi_gdf = coastsegmap.rois.gdf[coastsegmap.rois.gdf['id']==23]
single_roi_gdf

In [ ]:
shoreline_in_roi = gpd.clip(coastsegmap.shoreline.gdf, single_roi_gdf)
shoreline_in_roi

In [ ]:
shoreline_in_roi.iloc[-1:]["geometry"]

In [ ]:
shoreline_in_roi['geometry'][10]

In [ ]:
type(shoreline_in_roi['geometry'][10])

In [ ]:
np.array(shoreline_in_roi['geometry'][10])

In [ ]:
type(shoreline_in_roi['geometry'][21])

In [ ]:
shoreline_in_roi['geometry']

In [ ]:
exploded = shoreline_in_roi.explode()
exploded 

In [ ]:
np.array(shoreline_in_roi['geometry'][21])

In [ ]:
shoreline_in_roi['geometry'][21]

In [ ]:
shoreline_in_roi['geometry'].keys()

In [ ]:
# for index in shoreline_in_roi['geometry'].keys():
#     print(index)
#     print(type(index))

In [ ]:
    for k in shoreline_in_roi['geometry'].keys():
            #For each linestring portion of shoreline convert to lat,lon tuples
            shorelines.append(tuple(np.array(shoreline_in_roi['geometry'][k]).tolist()))

In [ ]:
coastsegmap.settings

In [ ]:
coastsegmap.rois.inputs_dict

In [ ]:
coastsegmap.ROI_layer

In [ ]:
coastsegmap.selected_ROI_layer.data

In [ ]:
for roi in coastsegmap.selected_ROI_layer.data['features']:
    print(roi)
    print(roi['properties'])

In [ ]:
def get_inputs_list(selected_roi_geojson,
    dates: list,
    sat_list: list,
    collection: str)->list[dict]:
    """get_inputs_list Returns a list of all download settings each of ROI.
        Sample download settings:
        {'polygon': roi["geometry"]["coordinates"],
        'roi_id':roi['id'],
        'dates': dates,
        'sat_list': sat_list,
        'sitename' : 'ID02022-10-04__21_hr_39_min03sec',(ex folder name)
        'filepath': filepath,
        'landsat_collection': collection}
    Args:
        selected_roi_geojson:dict
            A geojson dictionary containing all the ROIs selected by the user
        dates: list
            A list of length two that contains a valid start and end date
        collection : str
        whether to use LandSat Collection 1 (`C01`) or Collection 2 (`C02`).
        sat_list: list
            A list of strings containing the names of the satellite
    Returns:
        list[dict]: list of all download settings each of ROI
    """        
    date_str = generate_datestring()
    # filepath: directory where data will be saved
    filepath = os.path.join(os.getcwd(), 'data')
    # create unique sitenames using the date and index of ROI
    # for each site create dictionary with download settings eg. dates,sitename
    site_ids=np.arange(len(selected_roi_geojson['features']))
    sitenames=list(map(lambda x:'ID' + str(x) + date_str,site_ids))
    attributes = list(map(lambda x: { 'dates': dates,
                                        'sat_list': sat_list,
                                        'sitename' : x,
                                        'filepath': filepath,
                                        'landsat_collection': collection} ,sitenames))
    logger.info(f"attributes: {attributes}")
    inputs_list=list(starmap(combine_inputs,(zip(selected_roi_geojson['features'],attributes))))
    logger.info(f"inputs_list: {inputs_list}")
    del attributes,sitenames,site_ids
    if inputs_list == []:
        logger.error("Error: No ROIs were selected. Please click a valid ROI on the map")
        raise Exception("Error: No ROIs were selected. Please click a valid ROI on the map\n")
    logger.info(f"Images available: \n {inputs_list}")
    return inputs_list

In [ ]:
coastsegmap.selected_ROI_layer.data['features'][0]['properties']['id']

In [ ]:
'roi_id':roi['id'],

In [ ]:
    polygon = roi["geometry"]["coordinates"]
    polygon = SDS_tools.smallest_rectangle(polygon)
    inputs = {
    'polygon': polygon,
    'roi_id':roi['id'],

## Run Models on Imagery
1. Select if you want to use a GPU or not. By default you won't your computer's GPU
2. Select if you want to use CRF post-processing or not. By default CRF post-processing is not enabled
<br>**WARNING**: There is a bug with CRF post-processing and the models so you will need to enable CRF post-processing until it is fixed.
3. Select whether your data is the type RGB or MNDWI
4. Select the model you want to use
5. Select Ensemble or Best (Ensemble is recommended) 
6. 


In [ ]:
from src.coastseg.UI_models import UI_Models
models_ui = UI_Models()
models_ui.create_dashboard()